<table align="left">
  <td>
    <a href="https://colab.research.google.com/github/Insight-Sogang-Univ/insight-14th/blob/main/advanced/template/session05/assignment_languagemodel.ipynb" target="_parent">
      <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
    </a>
  </td>
</table>


# 🔍 이론 문제

### 1. SLM(통계적 언어모델)의 한계에 대해 설명하세요.

**[1번 문제 정답]**

***여기에 정답을 써주세요!*** \
[정답] 가지고 있는 데이터가 부족해서 언어를 정확하게 모델링하지 못하는 희소 문제


### 2. BERT에 대한 설명이 아닌 것은?

a) 사전 학습 모델

b) 양방향 문맥 이해

c) 트랜스포머의 디코더를 쌓아 올린 구조

d) 이후 RoBERTa와 ALBERT로 발전

**[2번 문제 정답]**

***여기에 정답을 써주세요!*** \
[정답] C. 트랜스포머의 인코더를 쌓아 올린 구조이다.

### 3. BERT와 GPT의 학습 방식 차이를 간단히 설명하세요.

**[3번 문제 정답]**

***여기에 정답을 써주세요!*** \
[정답] BERT는 문장 전체를 보고 빈칸을 맞푸는 방식이라면 GPT는 앞부분 단어만 보고 다음 단어를 생성하는 방식이다.

### 4. LLM이 지닌 한계와 RAG의 필요성에 대해 간단히 서술하세요.

**[4번 문제 정답]**

***여기에 정답을 써주세요!*** \
[정답] 사실과 다른 내용을 그럴듯하게 만들어 내는 환각 현상과 막대한 업데이트 비용이 필요하다는 한계가 있으며 RAG는 '모르면 물어보자'라는 개념을 도입하여 LLM의 한계를 극복한다.

### 5. LLM을 경량화한 모델인 sLM을 만들기 위해 사용되는 대표적인 기술 3가지를 적으세요.(단답형)

**[5번 문제 정답]**

***여기에 정답을 써주세요!*** \
[정답] 가지치기, 양자화, 지식 증류

# ✍ 언어 모델 과제

### 🧮 GPU 사용하기

In [1]:
import torch
import tensorflow as tf

# PyTorch GPU 상태
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"✅ [PyTorch] Using device: {device}")
if torch.cuda.is_available():
    print(f"GPU name: {torch.cuda.get_device_name(0)}")


# TensorFlow GPU 상태
print("\n✅ [TensorFlow] GPU Devices:")
print(tf.config.list_physical_devices('GPU'))

✅ [PyTorch] Using device: cuda
GPU name: Tesla T4

✅ [TensorFlow] GPU Devices:
[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


# 🔍 1. BERT

**BERT (Bidirectional Encoder Representations from Transformers)**
- 문장 분류(스팸 메일 탐지), 질의응답 시스템(챗봇, 검색 엔진), 번역(다국어 지원), 텍스트 요약(뉴스 요약) 등 다양한 NLP 작업에 사용
- 문맥을 양방향으로 이해하여 텍스트의 의미를 정밀하게 파악하며, 사전 학습된 모델을 기반으로 빠르게 응용 가능

### 1-1. 모델과 tokenizer 초기화
- tokenizer를 통해 문장을 토큰으로 나누고, 이를 정수 인덱스로 변환하여 모델이 이해할 수 있도록 함

In [2]:
from transformers import BertTokenizer, BertForMaskedLM
import torch

# BERT 모델과 tokenizer 초기화
tokenizer = BertTokenizer.from_pretrained('klue/bert-base') # base : 모델크기, (uncased : 소문자 학습)
model = BertForMaskedLM.from_pretrained('klue/bert-base') # base : 모델크기, (uncased : 소문자 학습)
# tokenizer : raw text를 개별 토큰으로 분리(Wordpiece). 토크나이저 사전에 따라 고유한 정수 인덱스(고정값)를 매핑함.

# tokenizer를 통해 생성된 정수 인덱스 : 텍스트를 모델이 읽을 수 있도록 숫자화
# Embedding vector : 단어의 의미적, 문맥적 특성을 모델링

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/289 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

config.json:   0%|          | 0.00/425 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/445M [00:00<?, ?B/s]

Some weights of the model checkpoint at klue/bert-base were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


### 1-2. 입력 텍스트 준비 및 토큰화
- [MASK] 토큰을 삽입한 문장을 설정하고, `tokenizer.tokenize()`로 텍스트 토큰화
- [MASK]를 활용하여 문맥 속에서 특정 단어를 추론하는 상황을 만들고, 이를 모델이 학습한 패턴과 비교하도록 함


In [3]:
# 테스트할 문장
text = "[CLS] 열여덟, 우리는 서로의 이름을 처음 불렀다. 그리고 스물 하나, 우린 [MASK]을 했다.[SEP]"

# 문장을 토큰으로 변환
tokenized_text = tokenizer.tokenize(text) # text(문장)을 토큰으로 변환해 tokenized_text에 저장합시다!
indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text) # 토큰을 index로 변환하는 코드!

print(tokenized_text) # 토큰이 출력됩니다.
print(indexed_tokens) # 인덱스가 출력됩니다.


['[CLS]', '열', '##여', '##덟', ',', '우리', '##는', '서로', '##의', '이름', '##을', '처음', '불렀', '##다', '.', '그리고', '스물', '하나', ',', '우린', '[MASK]', '을', '했', '##다', '.', '[SEP]']
[2, 1432, 2173, 3542, 16, 3616, 2259, 4084, 2079, 3934, 2069, 3790, 6895, 2062, 18, 3673, 10514, 3657, 16, 8983, 4, 1498, 1902, 2062, 18, 3]


### 1-3. MASK된 위치 확인
- [MASK] 토큰의 위치를 탐지해 해당 위치에서 모델이 단어를 예측하도록 지정합니다.


In [4]:
# 마스킹된 위치 찾기
masked_index = tokenized_text.index("[MASK]") # 토큰화된 결과물에서 index 메서드를 통해 [MASK]의 위치를 확인해봅시다!
print(masked_index)

20


### 1-4. 정수화된 토큰 인덱스를 tensor로 변환 후 모델 예측 실행
- 모델은 **텐서 데이터 구조**를 필요로 하므로, **입력값을 변환**하여 적합한 형식으로 만듦.
   - `torch.tensor(입력값 리스트 또는 입력값 ndarray)`를 통해 tensor 구조로 변환 가능!
- **역전파를 비활성화**하고, 모델이 **각 토큰에 대해 가능한 모든 단어 점수 출력**

In [5]:
# 토큰화된 텍스트 -> 정수인덱스(ID) -> Pytorch 텐서
indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text) # 토큰을 index로 변환하는 코드!
tokens_tensor = torch.tensor([indexed_tokens]) # 입력값(토큰 인덱스 리스트)을 tensor 구조로 변환해봅시다!

# 모델에 토큰 텐서를 전달하고 예측 실행
with torch.no_grad(): # 가중치 업데이트 없으므로 자동미분연산은 끔
    outputs = model(tokens_tensor) # 예측. 모델은 각 토큰위치에 대한 예측 결과를 반환.
    predictions = outputs[0] # logits (각 토큰위치에서 가능한 모든 토큰들의 원시점수)
print(predictions)

tensor([[[ -6.0337,   4.5526,  -5.6312,  ...,  -7.3959,  -7.4220,  -5.8337],
         [ -6.1762,   4.7585,  -7.3973,  ...,  -7.6209, -12.2124,  -6.0998],
         [ -7.4194,   3.9148,  -6.1517,  ...,  -6.8162,  -9.5421,  -3.0473],
         ...,
         [ -7.3761,   8.6972,  -5.4904,  ...,  -8.7202,  -9.1403,  -5.8566],
         [ -5.6347,  10.3884,  -4.3374,  ...,  -8.6900,  -7.4171,  -3.5043],
         [ -5.6652,  10.3407,  -4.4099,  ...,  -8.7521,  -7.4286,  -3.6681]]])


### 1-5. MASK된 토큰 예측
- [MASK] 위치에서 가장 높은 점수를 받은 토큰의 인덱스를 추출하고, 이를 단어로 변환

In [6]:
# 예측된 토큰 확인
predicted_index = torch.argmax(predictions[0, masked_index]).item() # masked_index에 들어갈 것으로 가장 확률이 높은 인덱스
predicted_token = tokenizer.convert_ids_to_tokens([predicted_index])[0]

# 예측된 토큰으로 마스크 채우기
tokenized_text[masked_index] = predicted_token
# 토큰화된 텍스트를 다시 문자열로 변환
filled_text = tokenizer.convert_tokens_to_string(tokenized_text[1:-1])

print("Original:", text)
print("Masked:", tokenized_text)
print("Predicted token:", predicted_token) ; print()

print("Filled sentence:", filled_text)

Original: [CLS] 열여덟, 우리는 서로의 이름을 처음 불렀다. 그리고 스물 하나, 우린 [MASK]을 했다.[SEP]
Masked: ['[CLS]', '열', '##여', '##덟', ',', '우리', '##는', '서로', '##의', '이름', '##을', '처음', '불렀', '##다', '.', '그리고', '스물', '하나', ',', '우린', '사랑', '을', '했', '##다', '.', '[SEP]']
Predicted token: 사랑

Filled sentence: 열여덟 , 우리는 서로의 이름을 처음 불렀다 . 그리고 스물 하나 , 우린 사랑 을 했다 .



### 1-6. MASK된 문장 복원
- 예측된 토큰으로 [MASK]를 대체하여 완성된 문장 생성

In [7]:
from transformers import BertTokenizer, BertForMaskedLM
import torch

# BERT 모델과 토크나이저 초기화
tokenizer = BertTokenizer.from_pretrained('klue/bert-base')
model = BertForMaskedLM.from_pretrained('klue/bert-base')

def fill_mask(input_text):
    # 텍스트를 토큰으로 변환
    tokenized_text = tokenizer.tokenize(input_text)
    indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)

    # 마스킹된 위치 찾기
    masked_index = tokenized_text.index("[MASK]") # 토큰화된 텍스트에서 마스킹된 위치를 찾아봅시다!

    # 토큰화된 텍스트 -> 정수인덱스(ID) -> Pytorch 텐서
    tokens_tensor = torch.tensor([indexed_tokens])

    # 모델에 토큰 텐서를 전달하고 예측 수행
    with torch.no_grad():
        outputs = model(tokens_tensor)
        predictions = outputs[0]

    # 예측된 토큰 확인
    predicted_index = torch.argmax(predictions[0, masked_index]).item() # 위 코드 참고! masked_index에 들어갈 것으로 가장 확률이 높은 인덱스
    predicted_token = tokenizer.convert_ids_to_tokens([predicted_index])[0]

    # 마스크를 채운 문장 반환
    tokenized_text[masked_index] = predicted_token # 토큰화된 텍스트의 마스킹된 위치(인덱스 사용)를 예측된 토큰으로 채워봅시다!
    return tokenizer.convert_tokens_to_string(tokenized_text[1:-1])

Some weights of the model checkpoint at klue/bert-base were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


예시 문장 만들어서 MLM을 사용해 보세요!

In [8]:
# 예시 1: "[CLS] 열여덟, 우리는 서로의 이름을 처음 불렀다. 그리고 스물 하나, 우린 [MASK]을 했다.[SEP]"
# 예시 2: "[CLS] 윤해민은 INSGIHT 학회를 정말 사랑한다. 그는 INSIGHT 학회를 위해 [MASK]를 했다.[SEP]"
# 이 예시를 참고해서 [CLS](맨 앞), [MASK](채울 부분), [SEP](맨 뒤)를 추가해서 여러분만의 예시 문장 만들어주세요
input_text = "[CLS] 윤해민은 INSGIHT 학회를 정말 사랑한다. 그는 INSIGHT 학회를 위해 [MASK]을 했다.[SEP]"
filled_text = fill_mask(input_text)
print("Filled sentence:", filled_text)

Filled sentence: 윤해민은 INSGIHT 학회를 정말 사랑한다 . 그는 INSIGHT 학회를 위해 헌신 을 했다 .


# 🔍 2. GPT, RAG, Langchain

## 2-1 라이브러리 설치

langchain 관련 라이브러리와, pdf 문서를 읽을수 있는 pypdf 라이브러리를 설치할게요!

In [9]:
# ⬇️ Colab에서 한 번만 실행
%pip install -qU "langchain>=0.3" langchain-community langchain-openai langgraph chromadb pypdf tiktoken


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.7/93.7 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 49.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.5/82.5 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 156.8/156.8 kB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.4/21.4 MB 42.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 328.9/328.9 kB 19.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 17.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 68.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 58.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.9/471.9 kB 33.1 MB/s eta 0:0

## 2-2. 키값 설정

## 2-3 pdf 문서 선택 (로드)


여러분들이 활용하고 싶은 아무 pdf 문서를 선택해보세요!

In [ ]:
from google.colab import files
import os, re, unicodedata

os.makedirs("/content/pdfs", exist_ok=True)
print("📥 업로드할 PDF 파일을 선택하세요 (여러 개 가능).")
uploaded = files.upload()

name_map = {}
for i, (orig_name, data) in enumerate(uploaded.items(), 1):
    nf = unicodedata.normalize("NFC", orig_name)
    base, ext = os.path.splitext(nf)
    # 한글/영문/숫자/밑줄/대시/점만 남기고 나머지는 _
    safe_base = re.sub(r"[^\w\u3131-\u318E\uAC00-\uD7A3.\-]+", "_", base).strip("._'‘’“”")
    # 파일명 너무 길면 잘라냄 (최대 ~140자 정도로)
    safe_base = safe_base[:120] if len(safe_base) > 120 else safe_base
    new_name = f"doc_{i}_{safe_base}{ext.lower()}"
    with open(os.path.join("/content/pdfs", new_name), "wb") as f:
        f.write(data)
    name_map[new_name] = orig_name

print("✅ 저장 완료 (Colab 파일명 → 원본명 매핑):")
for k, v in name_map.items():
    print(f"  {k}  <=  {v}")


📥 업로드할 PDF 파일을 선택하세요 (여러 개 가능).


## 2-4 문서 분할 후 벡터 DB에 저장
로딩한 문서를 분할 할 때,
- chunk_size는 `900`
- chunk_overlap은 `150`
- separators(구분자)는 `["\n\n", "\n", " ", ""]`
으로 설정하여 청킹해주세요!

In [ ]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import Chroma
import glob

# 1) PDF → Documents
pdf_paths = sorted(glob.glob("/content/pdfs/*.pdf"))
docs = []
for p in pdf_paths:
    try:
        loader = PyPDFLoader(p)
        docs.extend(loader.load())
    except Exception as e:
        print(f"⚠️ 로딩 실패: {p} -> {e}")

print(f"총 문서 조각(페이지 기준): {len(docs)}")

# 2) 문서 분할
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size= 900, # 청크 사이즈: 문서를 몇 개의 토큰 단위로 나눌 것인지를 정합니다.
    chunk_overlap= 150, # 청크 오버랩: 분할된 끝 부분에서 맥락이 이어질 수 있도록 일부를 겹쳐서(overlap) 분할하는 것이 일반적입니다.
    separators= ["\n\n", "\n", " ", ""], # 구분자: 엔터, 공백 기준으로 나눔
)
chunks = text_splitter.split_documents(docs)
print(f"총 청크 수: {len(chunks)}")

# 3) 벡터스토어 (영속 디렉토리 쓰면 런타임 재시작해도 유지)
persist_dir = "/content/chroma_pdf_db"
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")  # 비용 절약형
vectordb = Chroma.from_documents(
    documents=chunks,
    embedding=embeddings,
    persist_directory=persist_dir
)
retriever = vectordb.as_retriever(search_kwargs={"k": 5})
print("✅ 인덱싱 완료")


## 2-5 RA<U>Generate</U> : 생성 부분 선언

PROMPT 부분은 여러분들이 원하시는대로 입력해도 좋습니다!

🤥 영어로 입력하면 모델이 더 잘 이해한대요

In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

PROMPT = ChatPromptTemplate.from_template(
"""다음 컨텍스트(업로드한 PDF에서 추출됨)만을 바탕으로 질문에 한국어로 답하라.
불확실하면 모른다고 말하라. 핵심 근거를 요약하고, 각 근거 옆에 출처(파일/페이지)를 표시하라.

[질문]
{question}

[컨텍스트]
{context}
"""
)

rag_chain = (PROMPT | llm | StrOutputParser())


## 2-6 RAG 파이프라인 langgraph로 구성

벡터db에서 검색하는 것이 <U>Retrieve</U>AG이고,

검색 결과를 생성 단계에 던져주는 것이 R<U>Augment</U>G입니다!

In [ ]:
from typing import TypedDict, List
from langchain_core.documents import Document
from langgraph.graph import StateGraph, END

# format_docs의 역할은 문서를 포맷팅해서 모델에게 제공할 'context'를 정리하는 역할입니다!
def format_docs(docs):
    # 파일명/페이지를 메타데이터에 담아두는 PyPDFLoader 기본값 사용
    lines = []
    for i, d in enumerate(docs, 1):
        src = d.metadata.get("source", "")
        page = d.metadata.get("page", None)
        tag = f"{os.path.basename(src)}"
        if page is not None:
            tag += f" p.{page+1}"
        lines.append(f"[{i}] {tag}\n{d.page_content}")
    return "\n\n".join(lines)

# 그래프 선언
class QAState(TypedDict):
    question: str
    retrieved: List[Document]
    answer: str

def node_retrieve(state: QAState):
    q = state["question"]
    topk = retriever.invoke(q)
    return {"retrieved": topk}

def node_generate(state: QAState):
    q = state["question"]
    ctx = format_docs(state["retrieved"])
    ans = rag_chain.invoke({"question": q, "context": ctx})
    return {"answer": ans}

# --------- 그래프 구성 ---------
# 노드끼리 연결하기
workflow = StateGraph(QAState)
workflow.add_node("retrieve", node_retrieve) # 어떤 노드가 추가되어야 할까요?
workflow.add_node("generate", node_generate) # 어떤 노드가 추가되어야 할까요?

workflow.set_entry_point("retrieve")
workflow.add_edge("retrieve", "generate") # 어떤 노드와 어떤 노드가 연결되어야 할까요?
workflow.add_edge("generate", END)

app = workflow.compile()
print("✅ LangGraph 컴파일 완료")


## 2-7 실제 구동해보기

question에 여러분들이 입력하고 싶은 질문을 넣어보세요!

RAG 과제는 이렇게 간단하게 끝내겠습니다😁

In [ ]:
question = "이 논문의 서론 및 선행연구 부분을 범주화하여 정리해줘."
final = app.invoke({"question": question})
print("▼ 답변")
print(final["answer"])


In [ ]:
question = "이 논문에 대해서 요약해줘"
final = app.invoke({"question": question})
print("▼ 답변")
print(final["answer"])


In [ ]:
question = "이 논문에 있는 예시를 통해 음운적 특수성이 나타난 신어들에 대한 분석을 설명해줘."
final = app.invoke({"question": question})
print("▼ 답변")
print(final["answer"])


# 🔍 3. sLM

Copyright 2024 Google LLC.

## 3-1. sLM(Gemma) 로드
### sLM(small Language Model)의 대표 모델인 Gemma를 로드 해봅시다!

### (1) 사전 준비 사항
### 1. HuggingFace 로그인

https://huggingface.co/

이 링크로 들어가서 HuggingFace에 로그인합니다.(계정이 없다면 회원가입 해주세요!)

### 2. 토큰 발급받기

https://huggingface.co/settings/tokens

- 이 링크로 들어가서 'Create new token'을 클릭
- Token type: 'Read'로 해주세요.
- Token name: 아무거나 상관 없습니다.
- (⭐ 매우 중요!) 토큰 만드시고 **반드시 토큰 문자열 (hf_... 로 시작함)을 복사해주세요!**  
한 번만 볼 수 있습니다!

### 3. google colab에서 토큰 등록하기
- 왼쪽에 열쇠 모양 버튼(보안 비밀)을 눌러주세요.
- 이름: colab-gemma
- 값: 복사해둔 토큰 문자열(hf_...로 시작함)
- 노트북 액세스 : 스위치 키기(파란색)

### 4. gemma-2b-it 모델 사용 약관 동의

https://huggingface.co/google/gemma-2b-it

- (⭐ 매우 중요!) 반드시 2번 단계에서 토큰을 발급받았던 바로 그 Hugging Face 계정으로 로그인된 상태에서 위 링크로 들어가야 합니다.
- 링크로 들어가서 'Access Gemma on Hugging Face' 부분의 약관을 읽고 동의 버튼을 클릭해 주세요.
- "You have been granted access to this model"이라는 메시지가 뜨면 성공입니다.

### (2) 필요한 툴 설치

In [ ]:
!pip install -U bitsandbytes
!pip install --upgrade -q transformers huggingface_hub peft \
  accelerate bitsandbytes datasets trl

In [ ]:
# HuggingFace에 로그인합니다.

from huggingface_hub import login
from google.colab import userdata

# 따음표 안에 본인의 보안 비밀 키의 이름을 적습니다.(여기선 colab-gemma)
try:
    login(token=userdata.get("colab-gemma"))
except Exception as e:
    print(f"Error logging in to Hugging Face: {e}")
    print("Please make sure you have added your Hugging Face token to Colab secrets with the name 'colab-gemma'")

In [ ]:
# 모델을 로드합니다.
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

# 계산 부하를 줄이고 추론 속도를 높이기 위해 '양자화'합니다.
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True, bnb_4bit_quant_type="nf4", bnb_4bit_compute_dtype=torch.bfloat16
)

# sLM(gemma)를 Q&A, 요약 등에 활용하기 위해 '지시 튜닝(Instruction-Tuned) 버전의 모델을 사용합니다.
# "google/gemma-2b-it" 모델을 사용합니다.
model_id = "google/gemma-2b-it"

# 토크나이저를 로드합니다.
tokenizer = AutoTokenizer.from_pretrained(model_id)

# 모델을 로드합니다.
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=bnb_config, # 양자화(quantization)
    device_map={"":"cuda:0"} # Changed from "0" to "cuda:0"
)

# GPU를 사용하도록 설정합니다.
device = "cuda"

## 3-2. sLM 활용
- sLM은 LLM을 경량화한 모델이기 때문에 언어 모델의 기능인 질의응답(추론), 요약, 분류(감정분석) 등이 가능합니다.
- **속도는 비교적 빠르지만, 성능이 그닥 좋지 않다는 것을 확인하실 수 있습니다.**

### Gemma 활용 예시 (Common Use Cases)


### (1) 질의응답(Reasoning)

#### 원하는 질문을 한 번 넣어보세요!

In [ ]:
print("--- 예시 1: 질의응답 (Reasoning) ---")

# content : '~~' 따음표 안에 질문을 써주시면 됩니다.
# 하고 싶은 질문 아무거나 넣어보세요!

# "role": "user" 입니다.

chat = [
    # Gemma 모델의 채팅 형식에 맞춰 'role'과 'content'를 지정합니다.
    { "role": "user", "content": "AI가 회계 분야에서 어떻게 활용될 수 있는지 알려줘" }
]

# 1. 딕셔너리 리스트('chat')를 Gemma 모델이 이해할 수 있는 공식 프롬프트 문자열로 변환합니다.
prompt = tokenizer.apply_chat_template(chat, tokenize=False, add_generation_prompt=True)

# 2. 변환된 프롬프트 문자열을 모델이 입력받을 수 있는 토큰 ID 텐서로 변환하고, GPU('device')로 보냅니다.
inputs = tokenizer(prompt, return_tensors="pt").to(device)

# 3. 토큰화된 입력을 모델에 전달하여 텍스트 ""생성""을 시작합니다 (최대 256개의 새 토큰 생성)
# 생성을 위해 'generate' 함수를 사용합니다.
outputs = model.generate(**inputs, max_new_tokens=256) # **inputs에서 **는 빈칸이 아닙니다.

# 4. 'outputs'에는 '입력'과 '답변'이 모두 포함되어 있습니다.
# 따라서 원본 입력 토큰의 길이(입력, 'inputs.input_ids.shape[1]')를 기준으로, 그 이후에 '새로 생성된' 토큰(답변)만 추출합니다.
new_tokens = outputs[0][inputs.input_ids.shape[1]:]

# 5. 추출된 토큰 ID 리스트를 다시 사람이 읽을 수 있는 문자열로 ""디코딩""합니다.
# 디코딩을 위해 'decode' 함수를 사용합니다.
text = tokenizer.decode(new_tokens, skip_special_tokens=True)

# 6. 최종 생성된 답변 텍스트를 화면에 출력합니다.
print(text)

### (2) 요약 (Summarization)

#### 원하는 글을 넣어서 요약해 보세요!

In [ ]:
print("\n--- 예시 2: 요약 (Summarization) ---")

# text_to_summarize = """~~~""" 따음표 안에 요약하고 싶은 글을 넣어주시면 됩니다.""
# 요약하고 싶은 글 아무거나 넣어보세요!

text_to_summarize = """
코스피 석 달 동안 1,000p 올랐는데 환율 급등한 이유는?

[앵커]
최근 석 달 동안 국내 증시 코스피가 1,000p 안팎으로 오를 만큼 강세인데환율이 급등하는 현상이 나타나고 있습니다.원·달러 환율의 약세 이유와 주식 시장 동향을 최재민 해설위원과 함께 짚어보겠습니다. 어서 오십시오. 통상적으로 주식 시장이 괜찮으면 환율은 내려가는 게 정설인데지금은 환율이 급등하고 있거든요. 어떤 이유 때문일까요?

[기자]
그렇습니다. 주식 시장과 환율은 비슷하게 움직이는 게 통상적인데요. 그런데 최근에는 역설적이고 특이한 현상이 나타난다고 볼 수 있겠습니다. 주요 원인은 환율 상승의 성격과 주가 상승을 이끄는 주체가 다르기 때문으로 해석할 수 있을 것 같고요. 환율이 오르는 건 국내 요인보다는 미국 중심의 글로벌 요인이 더 크게 작용하고 있다고 봐야 될 것 같습니다. 미 연준이 물가 안정을 위해서 현재의 기준금리 인하를 더 늦출 가능성이 크다는 전망이 최근 우세한 상황이고 이 때문에 미국 국채 금리가 급등하고안전자산 선호 심리가 더해지면서 달러인덱스도 강세를 보이고 있습니다.

[앵커]
달러인덱스 얘기해 주셨는데 금융시장에서 미국 달러화의 가치를 측정하는 게 달러인덱스잖아요. 최근에 얼마나 강세인 건가요?

[기자]
유로와 엔, 파운드 등 주요 6개국 통화 그러니까 영국 파운드, 캐나다 달러 포함해서 6개 나라 대비 미국 달러화의 평균적인 가치를 측정한 지수가 달러인덱스인데 1973년 3월 기준점 100으로 설정되어 있습니다. 이 수치보다 높으면 달러 가치가 강세, 낮으면 약세를 의미하는 거거든요. 최근 석 달 전에는 98대 정도에서 움직이다가 9월 말에는 96대까지 내려갔습니다. 내려갔다가 지난달부터 오르기 시작해 99∼100대를왔다갔다하고 있는 상황입니다. 그런데 올해 초에는 110까지 올랐었는데그때도 그래프를 보면 알겠습니다만 그때도 원달러 환율은 1,460원∼1,480원이었는데 지난주 금요일 원·달러 환율은 7개월 만에 1,450원을 넘어서서 종가 기준으로 1458원에 근접했습니다. 그러니까 그때 달러인덱스가 강세일 때하고 비슷한 정도로 갔다는 거고요. 달러인덱스가 10p나 낮은데도원달러 환율은 그때와 비슷한 수준이니까상대적으로 원화가 그때보다 더 힘을 못 쓰고 있다고 보시면 될 것 같습니다. 원화 가치는 일주일 동안 2%나 급락했고요. 중요 통화국 가운데 절하율이 가장 큽니다. 또 한가지 주목할 것은 엔화 약세가 계속되고 있다는 점인데 시장에서 우려가 나오는 수준입니다. 1달러에 150엔대 중반에서 움직이고 있고요. 3년 전에 1달러에 150엔을 넘어섰는데 좀처럼 약세를 탈출할 기미를 보이지 않고 있습니다. 우리와 경쟁 관계에 있는 일본 엔화의 가치가 떨어지면 우리 수출 기업의 가격 경쟁력이 상대적으로 낮아져 원화 약세 압력이 더 커진다는 걸 의미한다고 보시면 될 것 같습니다.

[앵커]
그런데 연말에는 원·달러 환율이 더 오를 수도 있다, 이렇게 전망하는 전문가들도 있는 것 같더라고요.

[기자]
여전히 높은 변동성을 예상하는 게 중론이고요. 상단을 1,500원도 열어둬야 한다는 전망도 나오고 있습니다. 가장 큰 게 역시 미국의 금리 인하 여부인데요. 미국은 기준금리가 하단이 3.75%, 상단이 4%고요. 우리 기준금리는 2.5% 수준이거든요. 미국은 지난달 0.25%p 금리 인하를 시행했습니다. 올해는 미 연준의 FOMC 회의가 이제 한 차례만 남았는데우리 시각으로 다음 달 11일에 열립니다. 올해의 통화정책 방향을 마무리하고내년 금리 경로에 대한 중요한 단서를 제공하기 때문에 시장의 이목이 집중돼 있는데 최근 유럽과 중국의 경기 둔화 가능성이 커지면서 투자자들이 리스크가 큰 원화와 같은 신흥국 자산을 회피하고 안전한 자산으로 옮겨가고 있는 추세고요. 이 때문에 외환시장에서 달러의 희소성을 높여 원달러 환율을 끌어올리는 효과를 나타내고 있습니다.

[앵커]
지금까지는 글로벌 요인들을 설명해 주셨는데 그렇다면 원화가 약세인 국내적 요인도 있을까요?

[기자]
아무래도 유가증권시장에서 찾을 수 있을 것 같고요. 코스피가 4,000선을 내줬어요. 3953선에서 마무리됐고 금요일에만 72포인트 하락해서 4000선이 무너졌는데 가장 큰 원인이 외국인들이 주식을 팔고 있기 때문입니다. 주간 기준 외국인의 유가증권시장 순매도액이 역대 최대 수준이고요. 지난주 외국인의 코스피 시장 순매도액은 7조 2,640억으로 집계됐는데,직전 역대 1위 기록은 지난 2021년 8월 둘째 주에 기록한 7조450억 원이었습니다. 특히 지난 4일에는 외국인들이 2조 2,300억 원 가까이 내다 팔면서 4년 3개월 만에 최대치를 기록했고요. 증권가에서는 당분간 외국인의 매도세가 지속할 것으로 전망하고 있습니다. 원달러 상승하는 이유도 외국인의 투자심리를 위축하는 요소로 작용하고 있고 그런데 한국은행은 경기 침체와 높은 가계 부채 부담 때문에 금리를 올리기 어려운 딜레마에 처해 있습니다. 이는 한국은행이 환율 방어보다는 국내 경제를 우선할 수 있다는 신호를 줘서원화 약세를 부추길 수 있다는 지적입니다.

[앵커]
그리고 또 미 연방정부 셧다운이 40일 가까이 되지 않았습니까? 이 부분도 우리 경제나 금융시장에 영향을 미칠까요?

[기자]
단기 셧다운은 크게 영향을 미치지 않는데 하지만 장기적으로 미국 경제 성장 둔화와 소비 위축을 통해서 우리 금융 시장과 실물 경제 전반에 부정적인 파장을 미치는 악재로 작용할 가능성이 굉장히 크고요. 이미 그 악영향은 우리 경제에도 영향을 미치고 있다고 보는 게 맞을 것 같습니다. 셧다운 장기화 시 정부 기관의 업무가 중단되면서, 중단되면 예를 들면 중요한 고용보고서서나 국내총생산 같은 핵심 경제지표의 발표가 지연되거든요. 가장 중요한 게 미국의 통화정책인데 미 연준은 데이터를 기반으로 금리 정책을 결정합니다. 그렇게 되면 주요 지표를 확인할 수 없게 되면 통화정책 결정에 차질이 생기게 되고요. 이는 곧 시장의 불확실성을 더욱 확대하는 결과를 가져오게 됩니다. 미국 소비자 심리가 위축되면 우리의 주력 수출품인 자동차와 전자제품 같은 미국 수출 수요를 둔화시킬 위험도 있습니다.

[앵커]
셧다운이 한 달 넘은 상황인데 언제쯤 해결될 것으로 전망을 하시나요?

[기자]
장담할 수 있는 상황이 아니고요. 2018∼2019년의 35일인 역대 최장 기록을 이미 넘어섰습니다. 참고로 미국의 회계연도는 10월에 시작해서 이듬해 9월에 마무리되는데 트럼프 대통령이 셧다운을 정치적 수단으로활용하려는 의지가 강해서 과거의 셧다운보다 해결이 더 오래 걸릴 수 있다는 경고가 지배적입니다. 셧다운은 한국 증시의 핵심적인 불안 요인 가운데 하나이기도 하고요. 투자자, 그러니까 특히 외국인 투자자의 셀 코리아를 더욱 부추길 가능성도 있습니다.

[앵커]
그리고 얼마 전에 한미 관세협상 타결 소식이 있었잖아요. 연간 200억 달러씩 투자하기로 한 것도 원달러 환율에 영향을 미친다고 봐야겠죠?

[기자]
아무래도 환율 상승 압력으로 작용할 가능성이 크다고 봐야 할 것 같고요. 외환시장의 수급 불균형을 심화하는 구조적 요인으로 작용할 가능성이 큽니다. 단기적으로 관세라는 불확실성을 해소할 수 있지만, 장기적으로는 지속적이고 구조적인 상승 압력으로 작용할 것 같고요. 이 때문에 당분간은 1,200원이나 1,300원대 초반 환율은 기대하기 어렵다는 분석이 지배적입니다. 전문가들은 대미 투자에 따른 수급 부담이 겹쳐서 1,400원대 환율은 '뉴 노멀'로 자리 잡을 가능성이 크다고 우려하고 있습니다.

[앵커]
끝으로 이번 주 국내 주식 시장,전문가들은 어떻게 전망하고 있는지 설명해 주시죠.

[기자]
전반적으로 이번 주는 글로벌 불확실성이 극도로 높은 상황에서 변동성이 매우 클 것으로 전망하고 있고요. 낙폭 과대 기술주와 정책 수혜가 예상되는 종목을 선별적으로 접근하는 전략을 권고하고 있습니다. 그러니까 만약 이번 주에 셧다운 관련 타결 소식이 나오면단기적으로 가장 강력한 증시 반등 동력이 될 것으로 판단하고 있습니다.

[앵커]
알겠습니다. 오늘 말씀 여기까지 듣겠습니다. 지금까지 최재민 해설위원과 함께했습니다. 고맙습니다.
"""

# content : '~~~'에 요청사항을 적으시면 됩니다. 여기서는 위 글을 세 문장으로 요약해 달라고 요청했습니다.
# "role": "user" 입니다.

chat = [
    { "role": "user", "content": f"Could you summarize the following text in three sentence?\n\nText:\n{text_to_summarize}" }
]

# 여기부터는 각 사례 모두 코드가 동일합니다.

# 1. 딕셔너리 리스트('chat')를 Gemma 모델이 이해할 수 있는 공식 프롬프트 문자열로 변환합니다.
prompt = tokenizer.apply_chat_template(chat, tokenize=False, add_generation_prompt=True)

# 2. 변환된 프롬프트 문자열을 모델이 입력받을 수 있는 토큰 ID 텐서로 변환하고, GPU('device')로 보냅니다.
inputs = tokenizer(prompt, return_tensors="pt").to(device)

# 3. 토큰화된 입력을 모델에 전달하여 텍스트 ""생성""을 시작합니다 (최대 256개의 새 토큰 생성).
# 생성을 위해 'generate' 함수를 사용합니다.
outputs = model.generate(**inputs, max_new_tokens=256) # **inputs에서 **는 빈칸이 아닙니다.

# 4. 'outputs'에는 '입력'과 '답변'이 모두 포함되어 있습니다.
# 따라서 원본 입력 토큰의 길이(입력, 'inputs.input_ids.shape[1]')를 기준으로, 그 이후에 '새로 생성된' 토큰(답변)만 추출합니다.
new_tokens = outputs[0][inputs.input_ids.shape[1]:]

# 5. 추출된 토큰 ID 리스트를 다시 사람이 읽을 수 있는 문자열로 ""디코딩""합니다.
# 디코딩을 위해 'decode' 함수를 사용합니다.
text = tokenizer.decode(new_tokens, skip_special_tokens=True)

# 6. 최종 생성된 답변 텍스트를 화면에 출력합니다.
print(text)

### (3) 분류 (Classification)

원하는 문장을 넣고 감정 분석을 해보세요!

In [ ]:
print("\n--- 예시 3: 분류 (Classification) ---")

# content: '~~~' 따음표 안에 요청사항을 적으시면 됩니다.
# 여기서는 긍정, 부정, 중립으로 감정분석을 요청했습니다.

# \n\nText: 여기에 감정분석할 문장을 적어주시면 됩니다.
# 원하는 문장을 넣고 감정 분석을 해보세요!

# "role": "user" 입니다.

chat = [
    { "role": "user", "content": "Classify the text into neutral, negative, or positive. Generate only the class, nothing else.\n\nText: 너무 재미가 없다(영화 리뷰)" }
]

# 1. 딕셔너리 리스트('chat')를 Gemma 모델이 이해할 수 있는 공식 프롬프트 문자열로 변환합니다.
prompt = tokenizer.apply_chat_template(chat, tokenize=False, add_generation_prompt=True)

# 2. 변환된 프롬프트 문자열을 모델이 입력받을 수 있는 토큰 ID 텐서로 변환하고, GPU('device')로 보냅니다.
inputs = tokenizer(prompt, return_tensors="pt").to(device)

# 3. 토큰화된 입력을 모델에 전달하여 텍스트 ""생성""을 시작합니다 (최대 256개의 새 토큰 생성).
# 생성을 위해 'generate' 함수를 사용합니다.
outputs = model.generate(**inputs, max_new_tokens=256) # **inputs에서 **는 빈칸이 아닙니다.

# 4. 'outputs'에는 '입력'과 '답변'이 모두 포함되어 있습니다.
# 따라서 원본 입력 토큰의 길이(입력, 'inputs.input_ids.shape[1]')를 기준으로, 그 이후에 '새로 생성된' 토큰(답변)만 추출합니다.
new_tokens = outputs[0][inputs.input_ids.shape[1]:]

# 5. 추출된 토큰 ID 리스트를 다시 사람이 읽을 수 있는 문자열로 ""디코딩""합니다.
# 디코딩을 위해 'decode' 함수를 사용합니다.
text = tokenizer.decode(new_tokens, skip_special_tokens=True)

# 6. 최종 생성된 답변 텍스트를 화면에 출력합니다.
print(text)


#### sLM 활용사례 결과 요약

- sLM은 질의응답이나 요약은 어느 정도 수행하지만, 퀄리티가 아쉽습니다.
- 감정 분석에서 '너무 재미가 없다'를 중립으로 판단하는 것으로 보아 분류 작업에서도 한계가 나타납니다.


- 이렇게 범용적인 활용에서는 한계가 나타나지만, 미세조정(Fine-Tuning)을 통해 특정 작업에 특화하면 해당 작업에는 좋은 성능을 내면서도 여전히 경량화된 모델의 장점을 누릴 수 있는 것이 sLM의 특징입니다!

# 🤩🤩 수고하셨습니다 🤩🤩
⭐️⭐️ 2-2의 API KEY 셀을 지우고 과제 제출해주세요!! ⭐️⭐️